<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Modelo-1:" data-toc-modified-id="Modelo-1:-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Modelo 1:</a></span></li><li><span><a href="#FUNCIONES" data-toc-modified-id="FUNCIONES-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>FUNCIONES</a></span></li><li><span><a href="#ENTRENAMIENTO" data-toc-modified-id="ENTRENAMIENTO-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>ENTRENAMIENTO</a></span></li><li><span><a href="#TEST" data-toc-modified-id="TEST-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>TEST</a></span></li></ul></div>

# Modelo 1:
Utiliza un algoritmo XGBoost.   
El modelo utiliza datos economicos de la pelicula, su duracion, datos temporales sobre su estrenos (como su año, por ejemplo).
Tambien se ha encontrado importante la información sobre la productora o el cast de personajes.
Otra variable importante es si la pelicula originalmente se rueda en ingles.  

El modelo se ejecuta en una pipeline. Esta pipeline consiste en un reescalado robuto y luego el propio modelo.


In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Binarizer

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.preprocessing import RobustScaler
# from sklearn.preprocessing import MinMaxScaler

#Automcompletar rápido
%config IPCompleter.greedy=True

from xgboost import XGBRegressor


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

# FUNCIONES

In [24]:
def frecuencias_series(series, lista_posible, grafico=False):
    """series: serie con elementos en string separados por comas
        lista_posible: todas las posibles strings en series
        """
    aux = pd.Series( dtype='float64')
    for elemento in list(lista_posible):
    
        n = series.apply(lambda x:  elemento in x).sum()/2100*100
        aux = pd.concat([aux,pd.Series({elemento:n})] )
    if grafico:
        aux.sort_values().plot.barh()
    return aux.sort_values()


def lista_posibles(series):
    aux = series.map(lambda x: sorted([d['name'] for d in get_dictionary(x)]))
    generos = set()
    for f in aux:
        generos.update(f)
    return generos



def get_dictionary(s):
    try:
        d = eval(s)
    except:
        d = {}
    return d

def get_temp_var(df_cat):
    a = df_cat.release_date.dt.year
    b = df_cat.release_date.dt.month
    c = df_cat.release_date.dt.dayofweek
    d = df_cat.release_date.dt.quarter
    x = pd.concat([a,b,c,d],axis=1)
    x.columns = ['rd_year', 'rd_mes', 'rd_dia_semana', 'rd_cuatrimestre']
    return x


def conteo_genero(serie):
    aux = []
    for numero in [0,1,2]:
        x = serie.apply(lambda lista: lista.count(numero))
        aux.append(x)
    df_aux = pd.concat(aux,axis=1)
    df_aux.columns = ['cast_genero_0','cast_genero_1','cast_genero_2']
    return df_aux

# z = cat['cast'].map(lambda x: sorted([d['gender'] for d in get_dictionary(x)]))
# df_cast_genero_freq = conteo_genero(z)

In [31]:
def procesamiento(cat):
    cat = cat.copy()
    #ELIMINAR
    a_eliminar = ['belongs_to_collection','homepage','imdb_id','poster_path','tagline','spoken_languages','overview',]
    cat.drop(columns=a_eliminar,inplace=True)

    #IMPUTACION
    cat.genres.fillna('OTRO', inplace=True)
    cat.production_companies.fillna('OTRO',inplace=True)
    cat.production_countries.fillna('OTRO',inplace=True)
    cat.Keywords.fillna('OTRO',inplace=True)
    cat.cast.fillna('OTRO',inplace=True)
    cat.crew.fillna('OTRO',inplace=True)

    cat.release_date = pd.to_datetime(cat.release_date)



    # LISTA VARIABLES
    lista_vars =['genres','production_companies','production_countries','Keywords','cast','crew']
    no_transformadas = [v for v in cat.columns.tolist() if v not in lista_vars]
    no_transformadas

    #DF VARIABLES SIN JSONS
    aux = []
    for var in lista_vars:
        z = cat[var].map(lambda x: sorted([d['name'] for d in get_dictionary(x)])).map(lambda x: ','.join(map(str, x)))
        aux.append(z)


    df_limpios = pd.concat(aux,axis=1)



    #DF FRECUENCIAS DE VARIABLES LIMPIAS
    aux = []
    for column in df_limpios.columns:
        z = df_limpios[column].map(lambda x: len(x.split(',')))
        aux.append(z)
    df_freq = pd.concat(aux,axis=1)
    df_freq.columns = [var+"_freq" for var in df_freq.columns]
    df_freq

    # VARIABLES TIEMPO
    df_temp = get_temp_var(cat)
    df_temp['rd_year'] = np.where(df_temp['rd_year']>2020,df_temp['rd_year']-100,df_temp['rd_year'])

    # frecuencia genero
    z = cat['cast'].map(lambda x: sorted([d['gender'] for d in get_dictionary(x)]))
    df_cast_genero_freq = conteo_genero(z)    
    
    
    # UNIR TODOS LOS DATASETS
    df_cat = pd.concat([cat[no_transformadas],df_temp,df_limpios,df_freq,df_cast_genero_freq],axis=1)


    # VARIABLES NUEVA WINSORIZADO
    df_cat['bin_english'] = np.where(df_cat.original_language=='en',1,0) 
    df_cat['genres_freq'] = np.where(df_cat['genres_freq']>4,4,df_cat['genres_freq'])
    df_cat['production_companies_freq'] = np.where(df_cat['production_companies_freq']>4,4,df_cat['production_companies_freq'])
    df_cat['production_countries_freq'] = np.where(df_cat['production_countries_freq']>1,1,0)


    variable = 'Keywords_freq'
    minimo = 0
    maximo = 35
    df_cat[variable].clip(minimo,maximo, inplace=True)

    variable = 'cast_freq'
    minimo = 0
    maximo = 85
    df_cat[variable].clip(minimo,maximo, inplace=True)

    variable = 'crew_freq'
    minimo = 0
    maximo = 135
    df_cat[variable].clip(minimo,maximo, inplace=True)
    return df_cat

In [29]:
def post_procesamiento_m1(df_cat):
    
    #ELIMINAR COLUMNAS
    df_cat.drop(columns={'status','original_language'},inplace=True)
    
    # DF_OBJETOS
    df_object = df_cat.select_dtypes(include = 'object').copy()
    
    # VARIABLES NUMERICAS DE DF_CAT
    df_cat2 = df_cat.select_dtypes(exclude = 'object').copy()
    df_cat2.drop(columns=['release_date'],inplace=True)
    return df_cat2

In [32]:
def pre_procesamiento(df):
    df = df.copy()
    
    mean = df.runtime.mean()
    df.runtime.fillna(mean,inplace=True)
    
    variable = 'runtime'
    minimo = 70
    maximo = 200

    df[variable].clip(minimo,maximo, inplace=True)

    variable = 'budget'
    minimo = 0
    maximo = 200000000

    df[variable].clip(minimo,maximo, inplace=True)

    variable = 'popularity'
    minimo = 0
    maximo = 50

    df[variable].clip(minimo,maximo, inplace=True)
    
    return df

# ENTRENAMIENTO

In [36]:
# datos train
nombre_fichero_datos = 'train.csv'
ruta_completa = ruta_proyecto + '/02_Datos/01_Originales/' + nombre_fichero_datos

datos = pd.read_csv(ruta_completa)
datos

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,...,2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,...,8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,...,10/10/14,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,...,3/9/12,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,...,2/5/09,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2996,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,tt0109403,en,Chasers,Military men Rock Reilly and Eddie Devane are ...,9.853270,...,4/22/94,102.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It was supposed to be a routine prisoner trans...,Chasers,"[{'id': 378, 'name': 'prison'}, {'id': 572, 'n...","[{'cast_id': 2, 'character': 'Rock Reilly', 'c...","[{'credit_id': '52fe4494c3a368484e02ac7d', 'de...",1596687
2996,2997,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...",NaN,tt2364975,sv,Vi är bäst!,Three girls in 1980s Stockholm decide to form ...,3.727996,...,3/28/13,102.0,"[{'iso_639_1': 'sv', 'name': 'svenska'}]",Released,NaN,We Are the Best!,"[{'id': 1192, 'name': 'sweden'}, {'id': 4470, ...","[{'cast_id': 5, 'character': 'Bobo', 'credit_i...","[{'credit_id': '5716b72ac3a3686678012c84', 'de...",180590
2997,2998,NaN,65000000,"[{'id': 80, 'name': 'Crime'}, {'id': 28, 'name...",NaN,tt0116908,en,The Long Kiss Goodnight,"Samantha Caine, suburban homemaker, is the ide...",14.482345,...,10/11/96,120.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,What's forgotten is not always gone.,The Long Kiss Goodnight,"[{'id': 441, 'name': 'assassination'}, {'id': ...","[{'cast_id': 10, 'character': 'Samantha Caine ...","[{'credit_id': '52fe443a9251416c7502d579', 'de...",89456761
2998,2999,NaN,42000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",http://www

In [37]:
cat = pre_procesamiento(datos)
df_cat =  procesamiento(cat)
df_cat2 = post_procesamiento_m1(df_cat)

In [50]:
m1 = XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, enable_categorical=False,
                              gamma=0.7, gpu_id=-1, importance_type=None,
                              interaction_constraints='', learning_rate=0.015,
                              max_delta_step=0, max_depth=25,
                              min_child_weight=1,
                              monotone_constraints='()', n_estimators=100,
                              n_jobs=8, num_parallel_tree=1, predictor='auto',
                              random_state=0, reg_alpha=1, reg_lambda=1,
                              scale_pos_weight=1, subsample=1,
                              tree_method='exact', validate_parameters=1,
                              verbosity=None)

In [57]:
# TRAIN / TEST
y = df_cat2['revenue']
x= df_cat2.drop(columns=['revenue'])
var_finales = ['budget',
 'popularity',
 'runtime',
 'rd_year',
 'rd_mes',
 'rd_dia_semana',
 'rd_cuatrimestre',
 'genres_freq',
 'production_companies_freq',
 'production_countries_freq',
 'Keywords_freq',
 'cast_freq',
 'crew_freq',
 'cast_genero_0',
 'cast_genero_1',
 'cast_genero_2',
 'bin_english']
x = x[var_finales]

In [69]:
# ROBUST SCALER
var_rs = x.select_dtypes(include='number').columns.tolist()
rs = RobustScaler(quantile_range=(10.0, 90.0))

In [72]:
# PIPELINE
ct = make_column_transformer(
    (rs, var_rs),
    remainder='passthrough')



In [73]:
pipe_entrenamiento = make_pipeline(ct,m1)

In [74]:
pipe_entrenamiento.fit(x,y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('robustscaler',
                                                  RobustScaler(quantile_range=(10.0,
                                                                               90.0)),
                                                  ['budget', 'popularity',
                                                   'runtime', 'rd_year',
                                                   'rd_mes', 'rd_dia_semana',
                                                   'rd_cuatrimestre',
                                                   'genres_freq',
                                                   'production_companies_freq',
                                                   'production_countries_freq',
                                                   'Keywords_freq', 'cast_freq',
                                                   'crew_freq

# TEST

In [77]:
nombre_fichero_datos = "test.csv"
ruta_completa = ruta_proyecto + '/02_Datos/01_Originales/' + nombre_fichero_datos

datos = pd.read_csv(ruta_completa)


In [78]:
cat = pre_procesamiento(datos)
df_cat =  procesamiento(cat)
df_cat2 = post_procesamiento_m1(df_cat)


var_finales = ['budget',
 'popularity',
 'runtime',
 'rd_year',
 'rd_mes',
 'rd_dia_semana',
 'rd_cuatrimestre',
 'genres_freq',
 'production_companies_freq',
 'production_countries_freq',
 'Keywords_freq',
 'cast_freq',
 'crew_freq',
 'cast_genero_0',
 'cast_genero_1',
 'cast_genero_2',
 'bin_english']

In [96]:
def submit(df_cat2):
    prediction = pipe_entrenamiento.predict(df_cat2[var_finales])
    df_submit = pd.DataFrame({'revenue':prediction}, index=df_cat2.id)
    nombre_cat = "submit.csv"
    df_submit.to_csv(ruta_proyecto + '/03_Notebooks/03_Sistema/' + nombre_cat)
    
submit(df_cat2)